## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-06-10-45-00 +0000,wsj,How the Immigration Raid at Hyundai’s Factory ...,https://www.wsj.com/business/autos/hyundai-ice...
1,2025-09-06-10-12-33 +0000,bbc,Trumps says Venezuelan jets will be shot down ...,https://www.bbc.com/news/articles/cr70511v774o...
2,2025-09-06-10-02-51 +0000,nyt,Offer Accepted,https://www.nytimes.com/2025/09/06/briefing/of...
3,2025-09-06-10-00-42 +0000,wapo,Business Insider yanked 40 essays with suspect...,https://www.washingtonpost.com/business/2025/0...
4,2025-09-06-10-00-40 +0000,wapo,MAGA has a new cautionary tale: England,https://www.washingtonpost.com/politics/2025/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2460/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,trump,50
23,new,17
9,will,13
481,report,12
0,immigration,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
115,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...,95
76,2025-09-06-01-39-40 +0000,nypost,Kilmar Abrego Garcia faces deportation to tiny...,https://nypost.com/2025/09/05/us-news/kilmar-a...,86
124,2025-09-05-22-25-26 +0000,nyt,A Second Weak Jobs Report Challenges Trump’s C...,https://www.nytimes.com/2025/09/05/us/politics...,85
32,2025-09-06-09-00-00 +0000,wsj,Rep. Marjorie Taylor Greene is walking a tight...,https://www.wsj.com/politics/policy/marjorie-t...,83
109,2025-09-05-23-05-39 +0000,nypost,Barron Trump ‘doing a semester’ at different N...,https://nypost.com/2025/09/05/us-news/barron-t...,82


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
115,95,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...
179,65,2025-09-05-20-08-47 +0000,nyt,Elon Musk Could Become First Trillionaire Unde...,https://www.nytimes.com/2025/09/05/business/el...
105,49,2025-09-05-23-19-00 +0000,wsj,The IT unemployment rate fell to 4.5% in Augus...,https://www.wsj.com/articles/it-unemployment-f...
79,49,2025-09-06-01-20-10 +0000,nyt,South Koreans Are Swept Up in Immigration Raid...,https://www.nytimes.com/2025/09/05/us/georgia-...
225,46,2025-09-05-17-33-52 +0000,latimes,CBS' 'Face the Nation' will no longer edit tap...,https://www.latimes.com/entertainment-arts/bus...
162,44,2025-09-05-20-58-03 +0000,nypost,Top secret SEAL Team 6 killed North Korean civ...,https://nypost.com/2025/09/05/us-news/top-secr...
305,44,2025-09-05-11-25-38 +0000,nyt,Western Troops in Ukraine Before a Peace Deal ...,https://www.nytimes.com/2025/09/05/world/europ...
70,36,2025-09-06-02-17-09 +0000,nypost,Ex-FBI official tipped off Chinese firm doing ...,https://nypost.com/2025/09/05/us-news/ex-fbi-o...
113,34,2025-09-05-22-59-37 +0000,nypost,Andrew Cuomo touts charter school support — wh...,https://nypost.com/2025/09/05/us-news/andrew-c...
259,31,2025-09-05-15-39-10 +0000,nyt,"Angela Rayner, UK Deputy Prime Minister, Resig...",https://www.nytimes.com/2025/09/05/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
